## Importação das bibliotecas

In [67]:
import numpy as np
import pandas as pd
import plotly
import plotly.offline as py
import yfinance as yf
import datetime
import plotly.graph_objs as go
import openpyxl as xl
import os

## Sem esse comando os graficos não sao exibidos no notebook

In [68]:
from plotly.offline import plot, iplot
plotly.offline.init_notebook_mode(connected=True)

## Recebe os inputs do usuário e busca as cotações na base de dados do Yahoo

In [69]:
# Recebe o input do usuario com o nome da ação
stock = input('Digite o código da ação que deseja analisar')
stock = stock + '.SA'
# Recebe do usuário o valor das médias 
num_mm = int(input('Digite um valor númerico para a média móvel'))

# Recebe do usuário o valor das médias 
qtt_std = int(input('Digite a quantidade de desvio padrão da estratégia'))

# Define o lote padrão de 100 ações
capital = int(input('Digite o capital disponível para essa estratégia'))

# Define o periodo de busca do historico de preços
ini_date = datetime.date(2013,1,1)
end_date = datetime.date(2020,12,31)

df_stock_prices = yf.download(stock, start=ini_date, end=end_date, interval='1d')
df_stock_prices.reset_index(drop=False, inplace=True)

df_stock_prices.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,19.990000,20.209999,19.690001,19.690001,16.812572,30182600
1,2013-01-03,19.809999,20.400000,19.700001,20.400000,17.418812,30552600
2,2013-01-04,20.330000,20.620001,20.170000,20.430000,17.444433,36141000
3,2013-01-07,20.480000,20.670000,19.950001,20.080000,17.145580,28069600
4,2013-01-08,20.110001,20.230000,19.459999,19.500000,16.650339,29091300


## Cria as colunas de média móvel no Data Frame e preenche com zero os campos com 'NaN'

In [70]:
df_stock_prices['MM'] = df_stock_prices.Close.rolling(window=num_mm).mean()
df_stock_prices['SPREAD_MM'] = df_stock_prices['Close'] - df_stock_prices['MM']

df_stock_prices['Start'] = 0.00
df_stock_prices['Stop'] = 0.00
df_stock_prices['Profit'] = 0.00

df_stock_prices['MM'] = df_stock_prices['MM'].fillna(0)

# Excluindo do Data frame os registros com a média móvel zerada pois não dá pra fazer backtest nesses registros
df_stock_prices = df_stock_prices.loc[df_stock_prices['MM']!=0]
df_stock_prices.head(25)

,Date,Open,High,Low,Close,Adj Close,Volume,MM,SPREAD_MM,Start,Stop,Profit
20,2013-01-31,18.260000,18.330000,17.900000,18.080000,15.437849,33246400,19.573809,-1.493810,0.0,0.0,0.0
21,2013-02-01,18.139999,18.650000,18.120001,18.459999,15.762321,28860400,19.515238,-1.055239,0.0,0.0,0.0
22,2013-02-04,18.160000,18.350000,17.889999,18.000000,15.369541,32294300,19.400952,-1.400952,0.0,0.0,0.0
23,2013-02-05,17.280001,18.290001,17.260000,18.080000,15.437849,77332900,19.289048,-1.209048,0.0,0.0,0.0
24,2013-02-06,17.930000,18.000000,17.530001,17.600000,15.027998,30886100,19.170952,-1.570952,0.0,0.0,0.0
25,2013-02-07,17.750000,17.980000,17.320000,17.500000,14.942616,37066400,19.075714,-1.575714,0.0,0.0,0.0
26,2013-02-08,17.549999,17.780001,17.379999,17.740000,15.147533,31068100,18.983333,-1.243333,0.0,0.0,0.0
27,2013-02-13,17.950001,18.000000,17.730000,17.799999,15.198770,22328200,18.886667,-1.086667,0.0,0.0,0.0
28,2013-02-14,17.790001,17.900000,17.580000,17.670000,15.087768,25902900,18.781428,-1.111428,0.0,0.0,0.0
29,2013-02-15,17.610001,17.790001,17.530001,17.629999,15.053612,25001600,18.681905,-1.051905,0.0,0.0,0.0


## Plota o gráfico com o preço de fechamento e as médias móveis

In [71]:
close_prices = go.Scatter(
                            x = df_stock_prices['Date'],
                            y = df_stock_prices['Close'],
                            name = stock + 'Preço de Fechamento',
                            line = dict(color = '#330000'),
                            opacity = 1.0)

close_prices2 = go.Candlestick(
                                x = df_stock_prices['Date'],
                                open = df_stock_prices['Open'],
                                high = df_stock_prices['High'],
                                low = df_stock_prices['Low'],
                                close = df_stock_prices['Close'])

mm = go.Scatter(
                    x = df_stock_prices['Date'],
                    y = df_stock_prices['MM'],
                    name = 'Média Móvel',
                    line = dict(color = '#B2FF66'),
                    opacity = 1.0)

layout = go.Layout(
                    autosize=False,
                    legend_orientation='h',
                    width=1000,
                    height=400)

data_prices = [close_prices,mm]

fig = go.Figure(data=data_prices, layout=layout)

py.iplot(fig)

## Faz o backtest da estratégia 

In [72]:
status = False
start_price = 0
last_spread = 0
price_std = (df_stock_prices['SPREAD_MM'].std() *-1) * 0.5
price_value_std = price_std * qtt_std
drawdown = 0
drawdown2 = 0
lowest_price = 0

print(price_std)
print(price_value_std)
for row in df_stock_prices.itertuples():
    lote = capital / row.Close
    #print(lote)
    # Nesse primeiro cenário é quando não há operação em aberto e a condição do desvio padrão foi atendida
    if (row.SPREAD_MM < price_value_std) & (status == False):
        #print(row.Index)
        status = True
        df_stock_prices.at[row.Index, 'Start'] = row.Close
        start_price = row.Close
        lowest_price = row.Close
    # Na condição abaixo há uma operação em aberto porém ainda não houve o cruzamento de mm para encerrar a operação
    elif (row.SPREAD_MM < (price_value_std)) & (status == True):
        ##print(status)
        df_stock_prices.at[row.Index, 'Start'] = start_price
        status = True
        if row.Close < lowest_price:
            lowest_price = row.Close
    # A condição abaixo o cenário é o encerramento de uma operação em aberto
    # calculando profit/loss
    elif (row.Close > start_price) & (status == True):
        # print(start_price)
        df_stock_prices.at[row.Index, 'Profit'] = (row.Close - start_price) * lote
        df_stock_prices.at[row.Index, 'Stop'] = row.Close
        if row.Close < lowest_price:
            lowest_price = row.Close
        if drawdown != 0:
            drawdown2 = (start_price - lowest_price) / start_price
            if drawdown2 > drawdown:
                drawdown = drawdown2
                drawdown2 = 0
        else:
            drawdown = (start_price - lowest_price) / start_price
        lowest_price = 0
        status = False
        start_price = 0

#df_stock_prices.head(285)
print(drawdown)

-0.7570701433592025
-1.514140286718405
0.7656250116415322


In [73]:
# No periodo diario , MM de 50 periodos no gráfico diário e 3 desvio padrão a estratégia está sendo lucrativa
print('O resultado da estratégia é R$ %7.2f' % df_stock_prices['Profit'].sum())

O resultado da estratégia é R$ 3218.08


In [77]:
# Calcula o numero de entradas realizadas na estratégia
df_stock_prices_count = df_stock_prices.loc[df_stock_prices['Profit']!=0].count()
entry_count = df_stock_prices_count['Profit']

#Calcula o numero de entradas com lucro
df_stock_prices_count = df_stock_prices.loc[df_stock_prices['Profit']>0].count()
entry_count_profit = df_stock_prices_count['Profit']

#Calcula numero de entradas com prejuizo
df_stock_prices_count = df_stock_prices.loc[df_stock_prices['Profit']<0].count()
entry_count_loss = df_stock_prices_count['Profit']

#Calcular payoff
df_stock_prices_sum = df_stock_prices.loc[df_stock_prices['Profit']>0].sum()
entry_sum_profit = df_stock_prices_sum['Profit']

df_stock_prices_sum = df_stock_prices.loc[df_stock_prices['Profit']<0].sum()
entry_sum_loss = df_stock_prices_sum['Profit']

payoff = (entry_sum_profit / entry_count_profit) / (entry_sum_loss / entry_count_loss)
hit_rate = entry_count_profit / entry_count

#print('O número de entradas executadas foi de %7.0f' % entry_count)

3218.0751839239183
23
0.0
0
nan


In [75]:
import os
# Se o arquivo existe ele será excluido para ser criado novamente
if os.path.exists('spread_mm.xlsx'):
  os.remove('spread_mm.xlsx')
else:
  print("The file does not exist")

# Exporta o data frame para um arquivo no Excel
df_stock_prices.to_excel('spread_mm.xlsx', sheet_name='Spread_MM', index=False)

# Cria um resumo do backtest em uma nova guia do Excel
wb = xl.load_workbook('spread_mm.xlsx')

sheet = wb.create_sheet(index = 0 , title = "Resumo_Backtest") 

sheet.column_dimensions['A'].width = 30
sheet.column_dimensions['B'].width = 20
sheet['A1'] = 'Resumo do Backtest'
sheet['A2'] = 'Periodo avaliado:'
sheet['B2'] = str(ini_date) + ' até ' + str(end_date)
sheet['A3'] = 'Papel:'
sheet['B3'] = stock.upper()
sheet['A4'] = 'Capital:'
sheet['B4'] = capital
sheet['B4'].style = 'Comma'
sheet['A5'] = 'Média Móvel Utilizada:'
sheet['B5'] = num_mm
sheet['A6'] = 'Resultado Financeiro: R$'
sheet['B6'] = df_stock_prices['Profit'].sum()
sheet['B6'].style = 'Comma'
sheet['A7'] = 'Rentabilidade:'
sheet['B7'] = df_stock_prices['Profit'].sum() / capital
sheet['B7'].style = 'Percent'
sheet['A8'] = 'Número de entradas Executadas:'
sheet['B8'] = entry_count
sheet['A9'] = 'Número de operações Vencedoras:'
sheet['B9'] = entry_count_profit
sheet['A10'] = 'Número de operações Perdedoras:'
sheet['B10'] = entry_count_loss
#sheet['A11'] = 'O payoff da estratégia é:'
#sheet['B11'] = payoff
#sheet['A12'] = 'O drawdown máximo da estratégia é de:'
#sheet['B12'] = drawdown
#sheet['B12'].style = 'Percent'
#sheet['A13'] = 'A taxa de acerto da estratégia é de:'
#sheet['B13'] = hit_rate
#sheet['B13'].style = 'Percent'

wb.save('spread_mm.xlsx')